In [ ]:
import pandas as pd
import numpy as np

# A榜 读取数据
# train_Urbanization_rate = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\城镇化率.xlsx')
# train_Salary_level = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\工资水平.xlsx')

# train_Employment_Information = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\就业信息.xlsx')
# train_Employment_Information1 = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\就业信息.xlsx',sheet_name=1)
# train_Employment_Information1 = train_Employment_Information1.drop_duplicates(subset=['城市名称','年份'],keep='first')
# train_Employment_Information2 = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\就业信息.xlsx',sheet_name=2)

# train_Age_structure = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\年龄结构.xlsx')
# train_Age_structure = train_Age_structure.rename(columns={'城市':'城市名称'})
# train_population_density = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\人口密度.xlsx')
# train_Population_size = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\人口规模.xlsx')


# train_standard_living  = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\生活水平.xlsx')
# train_standard_living1  = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\生活水平.xlsx',sheet_name=1,header=0)
# train_standard_living2  = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\生活水平.xlsx',sheet_name=2,header=0)
# train_standard_living3  = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\生活水平.xlsx',sheet_name=3,header=0)
# train_standard_living4  = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\生活水平.xlsx',sheet_name=4,header=0)
# train_standard_living5  = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\生活水平.xlsx',sheet_name=5,header=0)

# B榜 读取数据
train_Urbanization_rate = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\B\城镇化率.xlsx')

train_Employment_Information = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\B\就业信息.xlsx')
train_Employment_Information1 = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\B\就业信息.xlsx',sheet_name=1)

train_Age_structure = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\B\年龄结构.xlsx')
train_Age_structure = train_Age_structure.rename(columns={'城市':'城市名称'})
train_Population_size = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\B\人口规模.xlsx')


train_standard_living  = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\B\生活水平.xlsx',header=1)
train_standard_living1  = pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\B\生活水平.xlsx',sheet_name=1,header=1)


In [ ]:
#生活水平表
##人均可支配收入
def life_level(life_df,name,m,n,x,y):
    dfs = []
    for i,j in enumerate(life_df['城市名称']):
        df=pd.DataFrame(np.zeros((m,n)))
        df.columns=['城市名称','年份',name]
        df['城市名称']=j
        df['年份']=[col for col in range(x,y)]
        df[name] = life_df.iloc[i,1:].tolist()
        dfs.append(df)

    life_df = pd.concat(dfs).reset_index(drop=True)
    return life_df

train_standard_living=life_level(train_standard_living,'income_percapita',8,3,2015,2023)
train_standard_living1=life_level(train_standard_living1,'Consumption_per',8,3,2015,2023)

In [ ]:
#合并
df_train = train_Population_size.merge(train_Employment_Information,on=['城市名称','年份'],how='left')
df_train = df_train.merge(train_Employment_Information1,on=['城市名称','年份'],how='left')

df_train = df_train.merge(train_Age_structure,on=['城市名称','年份'],how='left')

df_train = df_train.merge(train_standard_living,on=['城市名称','年份'],how='left')
df_train = df_train.merge(train_standard_living1,on=['城市名称','年份'],how='left')


#排序
df_train['城市名称'] = df_train['城市名称'].apply(lambda x:int(x[4:]))
df_train.sort_values(by='城市名称',inplace=True)
df_train['城市名称'] = df_train['城市名称'].apply(lambda x:'city'+str(x))

df_train = df_train.reset_index(drop=True)

In [ ]:
#构造训练集
#2010-2019 -> 标签2020
train_df = pd.DataFrame({'城市名称': df_train['城市名称'].unique().tolist()})
for i, d in enumerate(range(2010, 2020)):
    tmp = df_train[df_train['年份'] == d].copy().reset_index(drop=True)
    tmp.drop(['常住人口（万人）', '年份'], axis=1, inplace=True)
    tmp.columns = ['城市名称'] + [f'{c}_year_{i}' for c in tmp.columns[1:]]
    train_df = train_df.merge(tmp, on='城市名称')

#构造验证集
#2011-2020 -> 标签2021
valid_df = pd.DataFrame({'城市名称': df_train['城市名称'].unique().tolist()})
for i, d in enumerate(range(2011, 2021)):
    tmp = df_train[df_train['年份'] == d].copy().reset_index(drop=True)
    tmp.drop(['常住人口（万人）', '年份'], axis=1, inplace=True)
    tmp.columns = ['城市名称'] + [f'{c}_year_{i}' for c in tmp.columns[1:]]
    valid_df = valid_df.merge(tmp, on='城市名称')

#构造测试集
#2012-2021
test_df = pd.DataFrame({'城市名称': df_train['城市名称'].unique().tolist()})
for i, d in enumerate(range(2012, 2022)):
    tmp = df_train[df_train['年份'] == d].copy()
    tmp.drop(['常住人口（万人）', '年份'], axis=1, inplace=True)
    tmp.columns = ['城市名称'] + [f'{c}_year_{i}' for c in tmp.columns[1:]]
    test_df = test_df.merge(tmp, on='城市名称').reset_index(drop=True)    

#标签构造
#训练集标签(2020)
tmp = pd.DataFrame()
tmp = df_train[df_train['年份']==2020][['城市名称','常住人口（万人）']]
train_df = train_df.merge(tmp, on='城市名称', how='left')

#测试集标签(2021)
tmp = pd.DataFrame()
tmp = df_train[df_train['年份']==2021][['城市名称','常住人口（万人）']]
valid_df = valid_df.merge(tmp, on='城市名称', how='left')


#训练特征
drop_cols = [col for col in train_df if train_df[col].isna().sum()/len(train_df)>0.9]
features = [col for col in train_df.columns if col not in ['城市名称', '常住人口（万人）']+drop_cols]
train_df = train_df.reset_index(drop=True) 
len(features)

In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from lightgbm import early_stopping
from lightgbm import log_evaluation
from sklearn.metrics import mean_squared_error


xgb_model = XGBRegressor(objective='reg:squarederror',
                        eval_metric='rmse',
                        tree_method='hist',
                        n_estimators=200,
                        colsample_bytree=0.5,
                        gamma=0.75,
                        learning_rate=0.06,
                        max_depth=7,
                        min_child_weight=10,
                        early_stopping_rounds=20,
                        random_state=42).fit(train_df[features], train_df['常住人口（万人）'],
                                             eval_set=[(train_df[features], train_df['常住人口（万人）']),(valid_df[features],valid_df['常住人口（万人）'])],
                                            verbose=50)
#模型评估
xgb_pred = xgb_model.predict(valid_df[features])
xgb_score = mean_squared_error(valid_df['常住人口（万人）'], xgb_pred)

#模型重要性
feat_imp_df=pd.DataFrame()
feat_imp_df['feature'] = train_df[features].columns
feat_imp_df['imp'] = xgb_model.feature_importances_
feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
feat_imp_df['rank'] = range(feat_imp_df.shape[0])


print('score :',1.0/(1.0+xgb_score))

print(np.array(valid_df['常住人口（万人）']))
print(xgb_pred)

#模型预测
sub = pd.read_csv(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\sub\submission_sample.csv')
sub['pred'] = xgb_model.predict(test_df[features])

In [ ]:
import pandas as pd
#B
train_Population_size=pd.read_excel(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\B\人口规模.xlsx')

train_Population_size[train_Population_size['年份']==2022].sort_values(by='常住人口（万人）',ascending=False)

In [ ]:
train_Population_size[train_Population_size['年份']==2011].sort_values(by='常住人口（万人）',ascending=False).head(40)

In [ ]:
#保存结果
# train_Population_size = train_Population_size[train_Population_size['年份']==2021]

# train_Population_size['城市名称'] = train_Population_size['城市名称'].apply(lambda x:int(x[4:]))
# train_Population_size.sort_values(by='城市名称',inplace=True)
# train_Population_size['城市名称'] = train_Population_size['城市名称'].apply(lambda x:'city'+str(x))

# train_Population_size = train_Population_size.reset_index(drop=True)
# train_Population_size[train_Population_size['年份']==2021].reset_index(drop=True)

sub['2021_count'] = train_Population_size[train_Population_size['年份']==2021].reset_index(drop=True)['常住人口（万人）']
sub['人口变化率']=((sub['pred']-sub['2021_count'])/sub['2021_count'])
sub.loc[abs(sub['人口变化率'])>0.06, 'pred'] = sub[abs(sub['人口变化率'])>0.06]['2021_count']
sub['pred'] = sub['pred'].round(2)

sub.drop(columns=['2021_count','人口变化率'],inplace=True)
sub.to_csv(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\sub\sub.csv',index=False)

In [ ]:
sub.to_csv(r'C:\Users\86130\Desktop\ML算法竞赛\CCF\城市人口分析\sub\sub.csv',index=False)